# Ejercicio - Busqueda de Alojamiento en Airbnb.

Supongamos que somos un agente de [Airbnb](http://www.airbnb.com) localizado en Lisboa, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado `airbnb.csv` (en la carpeta data) donde tenemos información de todos los alojamientos de Airbnb en Lisboa.

In [11]:
import pandas as pd
airbnb = pd.read_csv("./data/airbnb.csv")

In [12]:
airbnb.head()

,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
0,6499,14455,Entire home/apt,Belém,8,5.0,2,1.0,57.0
1,17031,66015,Entire home/apt,Alvalade,0,0.0,2,1.0,46.0
2,25659,107347,Entire home/apt,Santa Maria Maior,63,5.0,3,1.0,69.0
3,29248,125768,Entire home/apt,Santa Maria Maior,225,4.5,4,1.0,58.0
4,29396,126415,Entire home/apt,Santa Maria Maior,132,5.0,4,1.0,67.0


En concreto el dataset tiene las siguientes variables:
- room_id: el identificador de la propiedad
- host_id: el identificador del dueño de la propiedad
- room_type: tipo de propiedad (vivienda completa/(habitacion para compartir/habitación privada)
- neighborhood: el barrio de Lisboa
- reviews: El numero de opiniones
- overall_satisfaction: Puntuacion media del apartamento
- accommodates: El numero de personas que se pueden alojar en la propiedad
- bedrooms: El número de habitaciones
- price: El precio (en euros) por noche

### Caso 1.

Alicia va a ir a Lisboa durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos darle 3 alternativas.

In [ ]:
import config as cfc
import csv
from restaurantLoop import *
from google.cloud import storage


def descarga_restaurantes():
    miApartamento = None
    listaApartamentos = []
    i = 0

    with open(cfc.csvpath) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')

        for row in reader:
            if i > 0:
                miApartamento = Apartment(pId=row[0], pCodPosta=row[40], pProvincia=row[39], pCiudad=row[38],
                                          pCodPais=row[43], pNombrePais=row[44], pLatitud=row[45], pLongitud=row[46])

                listaApartamentos.append(miApartamento)

            i += 1

    # Una vez tenemos la lista de apartamentos, la filtramos si así está configurado
    if cfc.filtroPais != '*':
        listaApartamentos = [x for x in listaApartamentos if x.codPais() == cfc.filtroPais]

    # Fichero
    filep = open('restaurants.csv', 'w')
    print(
        f"Id Apartamento\",\"Id Restaurante\",\"Nombre restaurante\",\"Url Restaurante\",\"Direccion\",\"Precio medio\",\"Valoracion\"",
        file=filep)

    # Obtenemos los datos de la app El Tenedor
    miRest = RestaurantLoop()

    for ap in listaApartamentos:
        miRest = RestaurantLoop()
        miRest.setData(pApartamento=ap, pFile=filep)  # Establecer url coordenadas todos los apartamentos

    miRest.empieza()  # Iniciar crawler
    filep.close()  # Cerrar fichero resultados


def sube_archivos():
    client = storage.Client.from_service_account_json('key.json')
    buckets = list(client.list_buckets())

    bucket = client.get_bucket(buckets[0])  # Cogemos el primer segmento de la lista

    # Subir fichero restaurantes
    blob = bucket.blob('restaurants.csv')
    blob.upload_from_filename('restaurants.csv')

    # Subir fichero apartamentos
    blob = bucket.blob('apartments.csv')
    blob.upload_from_filename(cfc.csvpath)


def mostrarMenu():
    print('Seleccionar opción:')
    print('1- Descargar restaurantes')
    print('2- Subir ficheros a Google Storage')
    print('3- Salir')


def main():
    salir = False
    print(chr(27) + "[2J")

    while not salir:
        mostrarMenu()
        opcion = str(input("Opción (1|2|3): "))

        if opcion == '3':
            salir = True
            break
        elif opcion == '1':
            descarga_restaurantes()
        elif opcion == '2':
            sube_archivos()
        else:
            print(chr(27) + "[2J")


if __name__ == "__main__":
    main()

### Caso 2

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente enfadado, ya que su hermana Clara ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Clara. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Clara son 97503 y 90387  respectivamente. Finalmente guardamos este dataframe como excel llamado "roberto.xls


### Caso 3

Diana va a Lisboa a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 50€ para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas *(room_type == Shared room)*, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.